<a href="https://colab.research.google.com/github/foochane/text-classification/blob/master/%E6%96%87%E6%9C%AC%E5%88%86%E7%B1%BB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 导入数据

In [1]:
#载入接下来分析用的库
import pandas as pd
import numpy as np
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize

Using TensorFlow backend.


In [2]:
data=pd.read_excel('data/复旦大学中文文本分类语料.xlsx','sheet1') 

In [3]:
data.head()

,分类,正文
0,艺术,﻿【 文献号 】1-2432\n【原文出处】出版发行研究\n【原刊地名】京\n【原刊期号】1...
1,艺术,﻿【 文献号 】1-2435\n【原文出处】扬州师院学报：社科版\n【原刊期号】199504...
2,艺术,﻿【 文献号 】1-2785\n【原文出处】南通师专学报：社科版\n【原刊期号】199503...
3,艺术,﻿【 文献号 】1-3021\n【原文出处】社会科学战线\n【原刊地名】长春\n【原刊期号】...
4,艺术,﻿【 文献号 】1-3062\n【原文出处】上海文化\n【原刊期号】199505\n【原刊页...


In [4]:
data.分类.unique()

array(['艺术', '文学', '哲学', '通信', '能源', '历史', '矿藏', '空间', '教育', '交通', '计算机',
       '环境', '电子', '农业', '体育', '时政', '医疗', '经济', '法律'], dtype=object)

In [5]:
# data['正文'][0]

In [6]:
import jieba
jieba.enable_parallel(64) #并行分词开启
data['文本分词'] = data['正文'].apply(lambda i:jieba.cut(i) )

Building prefix dict from the default dictionary ...
I0908 19:11:12.172982 139761629574976 __init__.py:111] Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
I0908 19:11:12.174695 139761629574976 __init__.py:131] Loading model from cache /tmp/jieba.cache
Loading model cost 0.749 seconds.
I0908 19:11:12.923791 139761629574976 __init__.py:163] Loading model cost 0.749 seconds.
Prefix dict has been built succesfully.
I0908 19:11:12.924884 139761629574976 __init__.py:164] Prefix dict has been built succesfully.


In [7]:
data['文本分词'] =[' '.join(i) for i in data['文本分词']]

In [8]:
# data['文本分词'][8111]

评测指标

In [9]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """对数损失度量（Logarithmic Loss  Metric）的多分类版本。
    :param actual: 包含actual target classes的数组
    :param predicted: 分类预测结果矩阵, 每个类别都有一个概率
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

用scikit-learn中的LabelEncoder将文本标签（Text Label）转化为数字(Integer)

In [10]:
lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(data.分类.values)

In [11]:
y[:10]

array([16, 16, 16, 16, 16, 16, 16, 16, 16, 16])

将数据分成训练和验证集

In [12]:
xtrain, xvalid, ytrain, yvalid = train_test_split(data.文本分词.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [13]:
print (xtrain.shape)
print (xvalid.shape)

(8324,)
(925,)


## 构建基础模型（Basic Models）¶
让我们先创建一个非常基础的模型。

这个非常基础的模型（very first model）基于 TF-IDF (Term Frequency - Inverse Document Frequency)+逻辑斯底回归（Logistic Regression）

In [14]:
def number_normalizer(tokens):
    """ 将所有数字标记映射为一个占位符（Placeholder）。
    对于许多实际应用场景来说，以数字开头的tokens不是很有用，
    但这样tokens的存在也有一定相关性。 通过将所有数字都表示成同一个符号，可以达到降维的目的。
    """
    return ("#NUMBER" if token[0].isdigit() else token for token in tokens)


class NumberNormalizingVectorizer(TfidfVectorizer):
    def build_tokenizer(self):
        tokenize = super(NumberNormalizingVectorizer, self).build_tokenizer()
        return lambda doc: list(number_normalizer(tokenize(doc)))

In [15]:
# 利用刚才创建的NumberNormalizingVectorizer类来提取文本特征，注意里面各类参数的含义，自己去sklearn官方网站找教程看

stwlist=[line.strip() for line in open('data/哈工大停用词表.txt',
'r',encoding='utf-8').readlines()]
tfv = NumberNormalizingVectorizer(min_df=3,  
                                  max_df=0.5,
                                  max_features=None,                 
                                  ngram_range=(1, 2), 
                                  use_idf=True,
                                  smooth_idf=True,
                                  stop_words = stwlist)

# 使用TF-IDF来fit训练集和测试集（半监督学习）
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['#NUMBER', 'lex', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [16]:
xtrain_tfv.shape

(8324, 913297)

In [17]:
xtrain_tfv[0].toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [18]:
#利用提取的TFIDF特征来fit一个简单的Logistic Regression 
clf = LogisticRegression(C=1.0,solver='lbfgs',multi_class='multinomial')
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))
#print(classification_report(predictions, yvalid))

logloss: 0.606 


/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [19]:
 accuracy = metrics.accuracy_score( clf.predict(xvalid_tfv),yvalid)
print ("Accuracy: ", accuracy)

Accuracy:  0.8908108108108108


In [20]:
# Classification report
print(metrics.classification_report(yvalid, clf.predict(xvalid_tfv),
                                    target_names=data['分类'].unique()))

              precision    recall  f1-score   support

          艺术       1.00      0.17      0.29         6
          文学       0.87      0.94      0.90       126
          哲学       0.95      0.92      0.94       102
          通信       0.00      0.00      0.00         5
          能源       0.79      0.72      0.76        47
          历史       0.00      0.00      0.00         5
          矿藏       0.00      0.00      0.00         6
          空间       0.00      0.00      0.00         3
          教育       0.92      0.87      0.89        52
          交通       1.00      0.20      0.33         5
         计算机       0.90      0.97      0.93       122
          环境       0.00      0.00      0.00         3
          电子       0.00      0.00      0.00         3
          农业       0.93      0.84      0.89        64
          体育       0.88      0.97      0.93       160
          时政       0.00      0.00      0.00         3
          医疗       0.81      0.91      0.85        74
          经济       0.92    

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## 使用Count Vectorizer+Logistic Regression 

In [21]:
ctv = CountVectorizer(min_df=3,
                      max_df=0.5,
                      ngram_range=(1,2),
                      stop_words = stwlist)

# 使用Count Vectorizer来fit训练集和测试集（半监督学习）
ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lex', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '１２', 'ｌｉ', 'ｚｘｆｉｔｌ'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [22]:
#利用提取的word counts特征来fit一个简单的Logistic Regression 

clf = LogisticRegression(C=1.0,solver='lbfgs',multi_class='multinomial')
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict_proba(xvalid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.751 


/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


## TFIDF+朴素贝叶斯模型

In [23]:
#利用提取的TFIDF特征来fitNaive Bayes
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict_proba(xvalid_tfv)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))


logloss: 0.970 


## SVM

In [24]:
#使用SVD进行降维，components设为120，对于SVM来说，SVD的components的合适调整区间一般为120~200 
svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)

#对从SVD获得的数据进行缩放
scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [25]:
# 调用下SVM模型
clf = SVC(C=1.0, probability=True) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.344 


## TFIDF+xgboost

In [26]:
# 基于tf-idf特征，使用xgboost
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict_proba(xvalid_tfv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.178 


## word counts + xgboost

In [27]:
# 基于tf-idf的svd特征，使用xgboost
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.365 


## 基于tf-idf的svd特征，使用xgboost


In [28]:
# 基于tf-idf的svd特征，使用xgboost
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_svd, ytrain)
predictions = clf.predict_proba(xvalid_svd)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.365 


## 再对经过数据标准化(Scaling)的tf-idf-svd特征使用xgboost

In [29]:
# 再对经过数据标准化(Scaling)的tf-idf-svd特征使用xgboost
clf = xgb.XGBClassifier(nthread=10)
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict_proba(xvalid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(yvalid, predictions))

logloss: 0.370 


## 网格搜索（Grid Search）

网格搜索是一种超参数优化的技巧。

In [30]:
mll_scorer = metrics.make_scorer(multiclass_logloss, 
                       greater_is_better=False, needs_proba=True)

In [31]:
#SVD初始化
svd = TruncatedSVD()
    
# Standard Scaler初始化
scl = preprocessing.StandardScaler()

# 再一次使用Logistic Regression
lr_model = LogisticRegression()

# 创建pipeline 
clf = pipeline.Pipeline([('svd', svd),
                         ('scl', scl),
                         ('lr', lr_model)])

In [32]:
param_grid = {'svd__n_components' : [120, 180],
              'lr__C': [0.1, 1.0, 10], 
              'lr__penalty': ['l1', 'l2']}

In [33]:
# 网格搜索模型（Grid Search Model）初始化
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

#fit网格搜索模型
model.fit(xtrain_tfv, ytrain)  #为了减少计算量，这里我们仅使用xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 12 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 15.4min
[Parallel(n_jobs=-1)]: Done   4 out of  24 | elapsed: 16.0min remaining: 80.2min
[Parallel(n_jobs=-1)]: Done   7 out of  24 | elapsed: 16.1min remaining: 39.1min
[Parallel(n_jobs=-1)]: Done  10 out of  24 | elapsed: 16.4min remaining: 23.0min
[Parallel(n_jobs=-1)]: Done  13 out of  24 | elapsed: 19.9min remaining: 16.9min
[Parallel(n_jobs=-1)]: Done  16 out of  24 | elapsed: 20.6min remaining: 10.3min
[Parallel(n_jobs=-1)]: Done  19 out of  24 | elapsed: 21.8min remaining:  5.7min
[Parallel(n_jobs=-1)]: Done  22 out of  24 | elapsed: 22.1min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed: 44.4min finished
/home/ubuntu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/ubu

Best score: -0.370
Best parameters set:
	lr__C: 1.0
	lr__penalty: 'l1'
	svd__n_components: 180


In [34]:
nb_model = MultinomialNB()

# 创建pipeline 
clf = pipeline.Pipeline([('nb', nb_model)])

# 搜索参数设置
param_grid = {'nb__alpha': [0.001, 0.01, 0.1, 1, 10, 100]}

# 网格搜索模型（Grid Search Model）初始化
model = GridSearchCV(estimator=clf, param_grid=param_grid, scoring=mll_scorer,
                                 verbose=10, n_jobs=-1, iid=True, refit=True, cv=2)

# fit网格搜索模型
model.fit(xtrain_tfv, ytrain)  # 为了减少计算量，这里我们仅使用xtrain
print("Best score: %0.3f" % model.best_score_)
print("Best parameters set:")
best_parameters = model.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done   3 out of  12 | elapsed:    6.1s remaining:   18.2s
[Parallel(n_jobs=-1)]: Done   5 out of  12 | elapsed:   15.4s remaining:   21.5s
[Parallel(n_jobs=-1)]: Done   7 out of  12 | elapsed:   15.9s remaining:   11.4s
[Parallel(n_jobs=-1)]: Done   9 out of  12 | elapsed:   16.0s remaining:    5.3s
[Parallel(n_jobs=-1)]: Done  12 out of  12 | elapsed:   16.3s finished


Best score: -0.873
Best parameters set:
	nb__alpha: 0.001


## 基于word2vec的词嵌入

In [35]:
X = data['文本分词']

In [36]:
X = [i.split() for i in X]

In [37]:
# 训练word2vec词向量:
import gensim

model = gensim.models.Word2Vec(X,min_count =5,window =8,size=100)   # X是经分词后的文本构成的list，也就是tokens的列表的列表
embeddings_index = dict(zip(model.wv.index2word, model.wv.vectors))

print('Found %s word vectors.' % len(embeddings_index))

Found 119775 word vectors.


In [38]:
# model['汽车']

In [39]:
def sent2vec(s):
    import jieba
    jieba.enable_parallel() #并行分词开启
    words = str(s).lower()
    #words = word_tokenize(words)
    words = jieba.lcut(words)
    words = [w for w in words if not w in stwlist]
    #words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            #M.append(embeddings_index[w])
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v / np.sqrt((v ** 2).sum())

In [40]:
# 对训练集和验证集使用上述函数，进行文本向量化处理
xtrain_w2v = [sent2vec(x) for x in tqdm(xtrain)]
xvalid_w2v = [sent2vec(x) for x in tqdm(xvalid)]

  0%|          | 0/8324 [00:00<?, ?it/s]/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]
  1%|          | 55/8324 [02:07<4:50:25,  2.11s/it]

OSError: [Errno 24] Too many open files